# VANI

**Initial notebook configuration**

In [19]:
%load_ext autoreload
# %load_ext snakeviz
# %load_ext jupyterflame

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
sys.path.append(os.path.dirname(os.getcwd()))

/usr/workspace/iopp/projects/vani-analysis-tool/vani-env/bin/python3
3.7.2 (default, Feb 26 2019, 08:59:10) 
[GCC 4.9.3]
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)


In [21]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Import libraries**

In [22]:
from IPython.display import HTML
from vani.analyzer import Analyzer, ClusterOptions, ClusterType

**Initialize VANI Analyzer**

In [23]:
%autoreload 2

# Initialize analyzer
co = ClusterOptions(cluster_type=ClusterType.Local)
vn = Analyzer(n_workers=8, cluster_options=co, debug=True)

/usr/workspace/iopp/projects/vani-analysis-tool/vani-env/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39880 instead
  f"Port {expected} is already in use.\n"


Dashboard link: http://127.0.0.1:39880/status


**Analyze logs**

In [24]:
%%time
%autoreload 2

# Analysis configuration
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

log_dir = "/p/lustre1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
# log_dir = "/p/lustre1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/lustre1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/lustre1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/lustre1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# Do the analysis
analysis = vn.analyze_parquet_logs(log_dir, max_depth=10)

Logs read (0.9825681960210204)
---------------
Job time: 33.702
I/O time: 26.025
Max duration: 1.521
Mean BW: 3.083
Total ranks: 1280.000
Total size: 1580.000
Total files: 1280.000
---------------
CPU times: user 1min 3s, sys: 7.91 s, total: 1min 11s
Wall time: 1min 47s


In [25]:
%%time
analysis.render_tree()

Analysis
└── Timeline Analysis
    ├── [0.000-33.702] Size=2 1580.00000 100.0% | Bandwidth=1 3.08268 100.0% | Parallelism=2 1280.00000 100.0% | Files=1 1280.00000 100.0%
    │   ├── [0.000-16.851] Size=1 561.69531 35.6% | Bandwidth=1 3.31496 107.5% | Parallelism=2 839.00000 65.5% | Files=1 839.00000 65.5%
    │   └── [16.851-33.702] Size=2 1018.30469 64.4% | Bandwidth=1 2.95366 95.8% | Parallelism=2 1200.00000 93.8% | Files=1 1200.00000 93.8%
    │       ├── [16.851-25.277] Size=2 958.79688 60.7% | Bandwidth=1 2.96894 96.3% | Parallelism=2 1180.00000 92.2% | Files=1 1180.00000 92.2%
    │       │   ├── [16.851-21.064] Size=1 435.54688 27.6% | Bandwidth=1 3.28148 106.4% | Parallelism=2 863.00000 67.4% | Files=1 863.00000 67.4%
    │       │   └── [21.064-25.277] Size=1 523.25000 33.1% | Bandwidth=1 2.70645 87.8% | Parallelism=2 842.00000 65.8% | Files=1 842.00000 65.8%
    │       └── [25.277-33.702] Size=1 59.50781 3.8% | Bandwidth=1 2.70663 87.8% | Parallelism=1 97.00000 7.6% | Files=

In [26]:
%%time
%autoreload 2
filter_group_node = analysis.children[0]
vn.save_filter_group_node_as_flamegraph(filter_group_node=filter_group_node, output_path="flamegraph-hacc.txt")

CPU times: user 2.77 ms, sys: 1.99 ms, total: 4.77 ms
Wall time: 7.4 ms


In [27]:
%%time
! chmod +x flamegraph.pl
! flamegraph.pl flamegraph-hacc.txt > flamegraph-hacc.svg

CPU times: user 113 ms, sys: 94 ms, total: 207 ms
Wall time: 678 ms


In [28]:
HTML(filename="flamegraph-hacc.svg")